# Import Library

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import ast

import warnings
warnings.filterwarnings('ignore')

#visualization libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 10000)

# Read Data

In [ ]:
transaction_new = pd.read_csv("../input/fashion-campus/transaction_new.csv")
cust = pd.read_csv("../input/fashion-campus/customer.csv")
products = pd.read_csv('../input/fashion-campus/product.csv', on_bad_lines='skip')
click_new = pd.read_csv('../input/fashion-campus/click_stream_new.csv')

In [ ]:
products

In [ ]:
transaction_new

In [ ]:
pd.merge(transaction_new, products[['id', 'articleType']], left_on='product_id', right_on="id", how='left')

# Labelling

In [ ]:
trx2 = transaction_new[transaction_new["payment_status"] == "Success"]
trx2.head()

In [ ]:
cust_trx= pd.merge(cust[["customer_id","first_join_date"]],trx2[["customer_id","created_at"]], on = "customer_id", how = "left")
cust_trx

In [ ]:
cust_trx["created_at"] = pd.to_datetime(cust_trx["created_at"])
cust_trx["first_join_date"] = pd.to_datetime(cust_trx["first_join_date"])

In [ ]:
cust_trx

In [ ]:
cust_trx['created_at'] = cust_trx['created_at'].dt.strftime('%Y-%m')
cust_trx['first_join_date'] = cust_trx['first_join_date'].dt.strftime('%Y-%m')

In [ ]:
cust_trx

In [ ]:
cust_trx.sort_values(by='first_join_date',ascending=False)

## Cust nvr trx

In [ ]:
cust_nvr_trx = cust_trx[cust_trx["created_at"].isna()]
cust_nvr_trx.shape

sebanyak 49.758 (unique) customer tidak pernah transaksi sukses, sisanya yakni 50.242 customer (unique) pernah transaksi sukses

## Cust trx

In [ ]:
cust_trx2 = cust_trx[~cust_trx["created_at"].isna()]
cust_trx2.shape

In [ ]:
cust_trx2

In [ ]:
# groupkan berdasarkan customer_id dan created_at
cust_trx4 = cust_trx2.groupby(["customer_id","created_at"])['first_join_date'].first().reset_index()

In [ ]:
cust_trx4

In [ ]:
cust_trx4["created_at"] = pd.to_datetime(cust_trx4["created_at"])

In [ ]:
size = cust_trx4.shape[0]
#temp = pd.DataFrame(columns=['A'])
a = []
for i in range (size-1) :
    # jika nilai pada baris ke i di kolom ke 0 == nilai pada baris selanjutnya kolom ke 0 (merupakan cust yg sama)
    if cust_trx4.iloc[i,0] == cust_trx4.iloc[i+1,0] :
        # maka (created_at pada bulan next - created_at pd bulan before) dibagi 1 bulan
        # ini untuk mencari jarak bulan transaksi sukses selanjutnya
        a.append((cust_trx4.iloc[i+1,1] - cust_trx4.iloc[i,1])/np.timedelta64(1,'M'))
    # jika nilai pada baris ke i di kolom ke 0 != nilai pada baris selanjutnya kolom ke 0 (merupakan cust yg berbeda)
    else :
        # maka (created_at pada bulan now - created_at pd bulan now) dibagi 1 bulan
        # alias hasilnya 0, alias itu transaksi terakhir si cust tsb, atau bisa jd dia cuma transaksi sekali
        a.append((cust_trx4.iloc[i,1] - cust_trx4.iloc[i,1])/np.timedelta64(1,'M'))

In [ ]:
diff = pd.DataFrame(a, columns = ["diff"])
diff = round(diff)
diff.head()

In [ ]:
# langsung aja concat kalo emang jumlah rowsnya sama dan yakin dia emang seharusnya ada di row itu

cust_trx5 = pd.concat([cust_trx4, diff], axis=1)
cust_trx5["diff"] = cust_trx5["diff"].fillna(0)
cust_trx5

disini, 1 berati jarak antara transaksi bulan sekarang dengan transaksi selanjutnya, berjarak 1 bulan. kalo 0 berati dah gada transaksi lagi (ga dipake), kalo 2,3dst berati ya jarak antara transaksi bulan ini dgn transaksi selanjutnya berjarak sesuai angka itu (churn)

In [ ]:
def renovated(df) : 
    if df["diff"] == 1 : 
        return 0
    elif df["diff"] == 0 :
        return 2
    else :
        return 1

    
cust_trx5["flagging"] = cust_trx5.apply(renovated, axis =1 )

disni berati, ketika jarak transaksi sekarang dgn selanjutnya adalah 1 bulan, dia akan dilabeli 0, maka dia terhitung transaksi rutin. kalo jaraknya 0 bulan, dia akan dilabeli 2, dan ketika jarak ke transaksi selanjutnya adalah di atas 1 bulan, dia akan dilabeli 1 (churn)

In [ ]:
cust_trx5

In [ ]:
final_flagging = cust_trx5[cust_trx5["flagging"] <= 1].drop(columns=['diff'])
final_flagging

nah disini, yang label 2 di drop, yang mana jika cust tsb cuma trans sekali selama ini atau itu adalah transaksi terakhirnya, maka dia tidak akan diikutkan ke dalam pembuatan untuk feature

In [ ]:
final_flagging["flagging"].value_counts(normalize = True)

- 1 adalah nasabah churn
- 0 adalah nasabah yang trx rutin

# Feature Engineering

feature engineering adalah proses untuk **menghasilkan/membuat feature** untuk modeling

# 1. Transaction

In [ ]:
transaction_new

In [ ]:
transaction_new.dtypes

In [ ]:
transaction_new["created_at"] = pd.to_datetime(transaction_new["created_at"], errors='coerce')
transaction_new["created_at"] = transaction_new["created_at"].dt.strftime('%Y-%m')
transaction_new["created_at"] = pd.to_datetime(transaction_new["created_at"], errors='coerce')

In [ ]:
transaction_new.dtypes

In [ ]:
transaction_new["promo_code"]=transaction_new["promo_code"].fillna(0)

In [ ]:
transaction_new

In [ ]:
def tn(df) : 
    if df["promo_code"] == 0 : 
        return 0
    else :
        return 1

    
transaction_new["promo_code2"] = transaction_new.apply(tn, axis =1 )

In [ ]:
transaction_new

In [ ]:
transaction_new2 = transaction_new.groupby(["customer_id","created_at"], as_index = False).agg({"promo_code2" : ["sum"], "quantity" : ["sum"]
                                                                                                ,"promo_amount" : ["sum"], "total_amount" : ["sum"]
                                                                               
})
transaction_new2.columns = list(map(''.join,transaction_new2.columns.values))

In [ ]:
transaction_new2

In [ ]:
transaction_new2.dtypes

In [ ]:
transaction_new3 = final_flagging.merge(transaction_new2[['customer_id','created_at', 'promo_code2sum','quantitysum','promo_amountsum','total_amountsum']], on=['customer_id','created_at'], how='left')

In [ ]:
transaction_new3

# 2. traffic source (device)

ini karena masing2 device pasti punya sessionnya masing-masing. tidak mungkin device yang berbeda memiliki session_id yang sama



In [ ]:
click_traffic = click_new.groupby(['session_id'])[['traffic_source']].first().reset_index()
click_traffic

In [ ]:
trx_click = pd.merge(trx2[['created_at','customer_id','session_id']], click_traffic[['session_id','traffic_source']], on='session_id')

trx_click['created_at'] = pd.to_datetime(trx_click["created_at"])
trx_click['created_at'] = trx_click['created_at'].dt.strftime('%Y-%m')
trx_click['created_at'] = pd.to_datetime(trx_click["created_at"])

trx_click2 = trx_click.groupby(['customer_id','created_at','traffic_source']).count().reset_index()
trx_click2

In [ ]:
# one hot encoding karena datanya merupakan kategori

trx_click3 = pd.get_dummies(trx_click2['traffic_source'])
trx_click3 = trx_click3.rename(columns = {
    'MOBILE' : 'mobile_used',
    'WEB' : 'web_used'
})
trx_click3

In [ ]:
trx_click4 = pd.concat([trx_click2, trx_click3], axis=1)
trx_click5 = trx_click4.groupby(['customer_id','created_at']).sum().reset_index()

trx_click5

In [ ]:
# di merge dengan label churn yang dah dibikin, di merge berdasarkan customer_id dan created_at

trx_click6 = final_flagging.merge(trx_click5.iloc[:,[0,1,3,4]], on=['customer_id','created_at'], how='left')
trx_click6

In [ ]:
# ini untuk menghitung persebaran data churn dari feature web_used

asess_1 = pd.crosstab(trx_click6["web_used"], trx_click6["flagging"], margins=True, margins_name="Total")
asess_1

In [ ]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195758)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_1["%nonevent"] = asess_1.apply(nonevent, axis = 1)
asess_1["%event"] = asess_1.apply(event, axis = 1)

In [ ]:
#Let's calculate WoE and save it to new a column WoE
asess_1["WoE"] = np.log(asess_1["%event"]/asess_1["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_1["IV"] = asess_1["WoE"]*(asess_1["%event"] - asess_1["%nonevent"])

#Let's sum IV
asess_1["IV"].sum() # Weak Predictor

karena IV nya 0.03, maka feature web_used merupakan weak predictor (namun tetap bisa digunakan untuk modelling). silakan lakukan hal yang sama untuk feature mobile_used dan feature lainnya

# Feature Selection
feature selection artinya kita **memilih feature** mana yang akan kita gunakan untuk modeling. feature yang dipilih merupakan feature yang memiliki IV > 0.02 (termasuk kategori weak, medium, serta strong predictor)

In [ ]:
# feature_all = trx_click6.copy()
feature_all = pd.merge(trx_click6, transaction_new3[['customer_id','created_at', 'promo_code2sum','quantitysum','promo_amountsum','total_amountsum']], on=['customer_id','created_at'], how='left')

# kalau udah punya banyak feature, pakai codingan di bawah ini untuk merge semua featurenya menjadi 1 dataframe
# feature_all = <nama_dataframe1>.merge(<nama_dataframe2>, on=['customer_id','created_at','flagging'])
# feature_all = feature_all.merge(<nama_dataframe3>, on=['customer_id','created_at','flagging'])


In [ ]:
feature_all

In [ ]:
feature_all["web_used"]=feature_all["web_used"].fillna(0)
feature_all["mobile_used"]=feature_all["mobile_used"].fillna(0)

In [ ]:
# code ini untuk mengeksport feature yang sudah kita pilih menjadi file csv agar memudahkan ketika run filenya

# feature_all.to_csv('final_feature.csv',index=False)

In [ ]:
X = feature_all.iloc[:,4:]
y = feature_all[["flagging"]]

In [ ]:
# ini adalah feature nya yang ingin kita cek IV nya
X

In [ ]:
# ini adalah labelnya
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.3)
print("Jumlah data training: ", X_train.shape)
print("Jumlah data testing: ", X_test.shape)

In [ ]:
# ini function untuk menghitung semua IV dari feature yang sudah kita buat tadi (dari proses feature engineering)

def calc_iv(df, feature, target, pr=False):
    
    lst = []

    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())

    iv = data['IV'].sum()

    return iv, data

In [ ]:
# iv_df = X_train.copy()
# iv_df["target"] = y_train

# features = iv_df.columns[:-1].tolist()

# iv_list = []
# for feature in features:
    # iv, data = calc_iv(iv_df, feature, 'target')
    # iv_list.append(round(iv,6))

# woe_df = pd.DataFrame(np.column_stack([features, iv_list]), 
                      # columns=['Feature', 'iv'])
# woe_df

diatas merupakan hasil IV dari feature mobile_used dan web_used. dapat dilihat bahwa hasil IV keduanya > 0.02 dan termasuk weak predictor. maka feature ini dapat kita gunakan untuk modeling

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver = "newton-cg", max_iter = 1000, C = 3.2)
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)

log_train = round(logreg.score(X_train, y_train) * 100, 2)
log_accuracy = round(accuracy_score(y_pred_log, y_test) * 100, 2)
log_f1 = round(f1_score(y_pred_log, y_test) * 100, 2)

print("Training Accuracy    :",log_train,"%")
print("Model Accuracy Score :",log_accuracy,"%")
print("\033[1m--------------------------------------------------------\033[0m")
print("Classification_Report: \n",classification_report(y_test,y_pred_log))

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(logreg, X_test, y_test)
plt.show()